In [1]:
import numpy as np

In [4]:
from math import sqrt

In [2]:
a = np.array([[0,0,0], [1,1,1]])
b = np.array([1,2,3])

In [3]:
a- b

array([[-1, -2, -3],
       [ 0, -1, -2]])

In [ ]:
class GSM:
    def __init__(self, mu, precision, scales, weights):
        self.mu = mu # 1 x dim
        self.precision = precision # identity matrix of dim
        self.scales = scales # 1 x nscales
        self.weights = weights # 1 x nscales
        self.mean = mu
        self.nscales = len(scales)
        self.ndims = len(mu)
        
    def eval(self, x):
        
        ndims   = self.ndims;
        nscales = self.nscales;
        ndata   = x.shape[0] # or maybe 1
        
        x_mu = x - self.mu
        
                norm_const = np.zeros((nscales, 1))
        maha = np.zeros((nscales, ndata))
        
        for j in range(1, nscales+1):
            norm_const[j] = np.sqrt(np.linalg.det(self.precision[j])) / ((2 * pi) ** (ndims / 2))
            maha[j,:] = np.sum(np.multiply(x_mu, self.precision[j] * x_mu), axis=0)
        
        exponent = np.exp(-0.5 * self.scales * maha)
        p = np.multiply(np.multiply(norm_const, self.weights), (np.power(self.scales, ndims/2))) * exponent
        
        p = np.sum(p, axis=0)
        
        return p
    
    def log_grad_x(self, x):
        ndims   = self.ndims;
        nscales = self.nscales;
        ndata   = x.shape[0] # or maybe 1
        
        x_mu = x - self.mu
        
        norm_const = np.zeros((nscales, 1))
        maha = np.zeros((nscales, ndata))
        
        for j in range(1, nscales+1):
            norm_const[j] = np.sqrt(np.linalg.det(self.precision[j])) / ((2 * pi) ** (ndims / 2))
            maha[j,:] = np.sum(np.multiply(x_mu, self.precision[j] * x_mu), axis=0)
            P_x_mu = self.precision[j] * x_mu
        
        exponent = np.exp(-0.5 * self.scales * maha)
        y = np.multiply(np.multiply(norm_const, self.weights), (np.power(self.scales, ndims/2))) * exponent
        y = y / np.sum(y, axis=0) # or maybe 1
        
        g = np.zeros([ndims, ndata])
        
        for s in range(1, nscales):
            g = g - self.scales[s] * P_x_mu * y
            
        return g

    def log_grad_weights(self, x):
        ndims   = self.ndims;
        nscales = self.nscales;
        ndata   = x.shape[0] # or maybe 1
        
        x_mu = x - self.mu
        
        norm_const = np.zeros((nscales, 1))
        maha = np.zeros((nscales, ndata))
        
        for j in range(1, nscales+1):
            norm_const[j] = np.sqrt(np.linalg.det(self.precision[j])) / ((2 * pi) ** (ndims / 2))
            maha[j,:] = np.sum(np.multiply(x_mu, self.precision[j] * x_mu), axis=0)
        
        exponent = np.exp(-0.5 * self.scales * maha)
        y = np.multiply(np.multiply(norm_const, self.weights), (np.power(self.scales, ndims/2))) * exponent
        
        invld_arr = np.sum(y, axis = 0)
        invld = np.where(invld_arr <=0)
        y[:invld] = []
        
        gamma = y / np.sum(self.weights * y ,axis=0)
        g = np.sum(gamma, axis=1) - ndata
        g = np.reshape(g, self.weights.shape)
        
        return g
    
    def sample(self, nsamples):
        ndims = len(self.mean)
        nscales = len(self.weights)
        self.weights = self.weights[::-1]
        
        cw = np.cumsum(self.weights, axis=0)
        
        mat_1 = np.matlib.repmat(np.random.rand(1, nsamples), nscales, 1)
        mat_2 = np.matlib.repmat(cw, 1, nsamples)
        mix_comp = np.sum(mat_1 <= mat_2, axis=0)
  
        # Whitening transform of precision
    
        tmp = np.zeros((ndims, nsamples))
        r = np.random.randn(ndims, samples)
        
        for j in range(1, nscales+1):
            tmp2 = np.linalg.lstsq(np.linalg.cholesky(self.precision[j]), r)
            idx = np.matlib.repmat(mix_comp == j, ndims, 1)
            tmp[idx] = tmp2[idx]
  
        mat_3 = np.matlib.repmat(np.sqrt(self.scales[mix_comp], ndims, 1))
        y = np.matlib.repmat(self.mean, 1, nsamples) + np.divide(tmp, mat_3)
        
        return y
    
    
    def z_distribution(self, x):
        ndims = self.ndims
        nscales = self.nscales
        ndata = x.shape[0] # Or maybe index 1
        
        x_mu = x - self.mu
        
        norm_const = np.zeros((nscales, 1))
        maha = np.zeros((nscales, ndata))
        
        for j in range(1, nscales+1):
            norm_const[j] = np.sqrt(np.linalg.det(self.precision[j])) / ((2 * pi) ** (ndims / 2))
            maha[j,:] = np.sum(np.multiply(x_mu, self.precision[j] * x_mu), axis=0)
            
        exponent = np.exp(-0.5 * self.scales * maha)
        y = np.multiply(np.multiply(norm_const, self.weights), (np.power(self.scales, ndims/2))) * exponent
        
        p = y / np.sum(y, axis=0)
        
        return p
    
    
    def em(self, x, niters):
        ndims = self.ndims
        nscales = self.nscales
        ndata = x.shape[0] # Or maybe index 1
        
        x_mu = x - self.mu
        
        norm_const = np.zeros((nscales, 1))
        maha = np.zeros((nscales, ndata))
        
        for j in range(1, nscales+1):
            norm_const[j] = np.sqrt(np.linalg.det(self.precision[j])) / ((2 * pi) ** (ndims / 2))
            maha[j,:] = np.sum(np.multiply(x_mu, self.precision[j] * x_mu), axis=0)
            
        for i in range(1, niters+1):
            exponent = np.exp(-0.5 * self.scales * maha)
            y = np.multiply(np.multiply(norm_const, self.weights), (np.power(self.scales, ndims/2))) * exponent
            
        invld_arr = np.sum(y, axis = 0)
        invld = np.where(invld_arr <=0)
        y[:invld] = 1
        
        gamma = np.divide(y, np.matlib.repmat(np.sum(y, axis=0), nscales, 1))
        self.weights = np.mean(gamma, axis=1)

        

In [15]:
np.zeros([2,2]).shape

(2, 2)

In [16]:
np.array([1,2,3,4])[::-1]

array([4, 3, 2, 1])

In [17]:
np.cumsum(np.array([1,2,3,4]),axis=0)

array([ 1,  3,  6, 10])